In [38]:
import pandas as pd
import numpy as np
import json
import random
import itertools
import ast
from collections import defaultdict
%matplotlib inline

In [39]:
dump_fpath = 'data/declarations.json'

with open(dump_fpath) as f:
    declarations = json.loads(f.read())

In [ ]:
random.choice(declarations)

{'bonds': [],
 'incomes': [{'comment': '', 'relative': None, 'size': 879178.51},
  {'comment': '',
   'relative': {'id': 2, 'name': 'Супруг(а)'},
   'size': 724000.0}],
 'main': {'document_type': {'id': 1, 'name': 'Антикоррупционная декларация'},
  'office': {'id': 3954,
   'name': 'ГУФСИН Самарская область',
   'post': 'начальник ФКУ ОКС ГУФСИН России по Самарской области',
   'region': {'id': 77, 'name': 'Самарская область'},
   'type': {'id': 7, 'name': 'Федеральный, без региональной структуры'},
   'url': 'http://www.63.fsin.su/'},
  'party': None,
  'person': {'family_name': 'Жарков',
   'given_name': 'И.Н.',
   'id': 34273,
   'name': 'Жарков И.Н. ',
   'patronymic_name': ''},
  'year': 2013},
 'real_estates': [{'comment': '',
   'country': 'Россия',
   'name': '',
   'own_type': {'id': 1, 'name': 'В собственности'},
   'region': None,
   'relative': None,
   'share': 0.1111111111111111,
   'share_type': {'id': 2, 'name': 'Долевая собственность'},
   'square': 42.7,
   'type': {'

Total declarations:

In [41]:
len(declarations)

90864

In [45]:
columns = ['document_type',
            'party',
            'person_id', 'person_name', 'year', 
           'office_id', 'office_name', 
           'office_post',
           'office_type_id', 'office_type_name',
           'office_region_id', 'office_region_name',
           'income_personal', 'income_relatives',
           'savings',
           'real_estate_amount', 'real_estate_squares']

rows = []
for dec in declarations:
    row = [
        dec['main']['document_type']['name'],
        dec['main']['party']['name'] if dec['main']['party'] else None,
        dec['main']['person']['id'],
        dec['main']['person']['name'],
        dec['main']['year'],
        dec['main']['office']['id'],
        dec['main']['office']['name'],
        dec['main']['office']['post'],
        dec['main']['office']['type']['id'] if dec['main']['office'].get('type') else None,
        dec['main']['office']['type']['name'] if dec['main']['office'].get('type') else None,
        dec['main']['office']['region']['id'] if dec['main']['office'].get('region') else None,
        dec['main']['office']['region']['name'] if dec['main']['office'].get('region') else None,
    ]
    income_personal_sum = sum([inc['size'] for inc in dec['incomes'] if inc['relative'] is None])
    income_relatives_sum = sum([inc['size'] for inc in dec['incomes'] if inc['relative'] is not None])
    savings_sum = sum([float(sav.split('руб.')[0].replace(',', '.').replace(' ', '')) for sav in dec['savings']])
    real_estate_amount = len(dec['real_estates'])
    real_estate_squares_sum = sum([(estate['square']  or 0) for estate in dec['real_estates']])
    row += [income_personal_sum, income_relatives_sum]
    row += [savings_sum, real_estate_amount, real_estate_squares_sum]
    rows.append(row)
len(rows)

90864

In [46]:
df = pd.DataFrame(rows, columns=columns)
df = df.sort_values(by=['person_id', 'year'])
df.head()

,document_type,party,person_id,person_name,year,office_id,office_name,office_post,office_type_id,office_type_name,office_region_id,office_region_name,income_personal,income_relatives,savings,real_estate_amount,real_estate_squares
30358,Предвыборная декларация,КПРФ,8,Зюганов Геннадий Андреевич,1998,14,Государственная Дума,Кандидат в депутаты государственной думы от из...,7,"Федеральный, без региональной структуры",NaN,None,124154.00,0.00,0.00,1,150.0
31970,Предвыборная декларация,КПРФ,8,Зюганов Геннадий Андреевич,1999,449,Президент Российской Федерации,Кандидат на должность президента РФ,7,"Федеральный, без региональной структуры",NaN,None,495443.00,10804.00,0.00,3,252.7
127,Предвыборная декларация,КПРФ,8,Зюганов Геннадий Андреевич,2006,14,Государственная Дума,,7,"Федеральный, без региональной структуры",NaN,None,1257784.00,0.00,141740.91,1,167.4
1109,Предвыборная декларация,КПРФ,8,Зюганов Геннадий Андреевич,2006,449,Президент Российской Федерации,кандидат на пост Президента РФ,7,"Федеральный, без региональной структуры",NaN,None,3445291.61,145376.87,400490.32,2,281.3
5520,Антикоррупционная декларация,КПРФ,8,Зюганов Геннадий Андреевич,2009,14,Государственная Дума,Член комитета Государственной Думы,7,"Федеральный, без региональной структуры",NaN,None,2075542.33,102279.19,0.00,3,448.7


In [47]:
df.to_csv('data/declarations_raw.csv', index=False)